In [ ]:
import numpy as np
import scipy as sp
import scipy.signal as sig
import altair as alt
import pandas as pd
from pathlib import Path  # pathlib is seriously awesome!
import sys
import sklearn.decomposition as sld

In [ ]:
DRS_file=pd.read_csv('All_RSC_Data.csv')
rs_file=pd.ExcelFile('2023-07-14_random_sample.xlsx')
#rs stands for random sample
rs_sheet_info=pd.read_excel(rs_file,'samples-354-U1449_U1450_U1451_U')
rs_sheet_lith=pd.read_excel(rs_file,'random_sample')
#rs_sheet = the specific sheet using to merge
MiddleDepth=(rs_sheet_info['Top depth CSF-A (m)']+rs_sheet_info['Bottom depth CSF-A (m)'])/2
rs_sheet_info.insert(11,'depth-m',MiddleDepth)
#calculating middle depth from top and bottom depth of core section and inserting values into sheet as new column
DRS_data=pd.merge(rs_sheet_info,DRS_file,on=['Site','Hole','depth-m'])
DRS_data=pd.merge(DRS_data,rs_sheet_lith,left_on=['ID'],right_on=['name'],how='left')
DRS_data.to_csv('DRS_data.csv',index=False)

In [ ]:
raw_data=pd.read_csv('DRS_data.csv') # <--Replace the folder and file names with the merge results csv file 

coercivity=pd.read_csv('2023-09-29_irm_gradient.csv')
coercivity.drop(coercivity[coercivity['max_d2']>2000].index, inplace=True)
coercivity['depth-m']=coercivity['depth-m']-1+0.01
coercivity['site']=coercivity.site+coercivity.hole
coercivity=coercivity.drop('hole', axis=1)

hysteresis=pd.read_csv('2023-09-29_hysteresis_parameters_by_lithology.csv')
hysteresis['depth-m']=hysteresis['top depth csf-a (m)']+0.01
hysteresis['site']=hysteresis.site+hysteresis.hole
hysteresis=hysteresis.drop('hole', axis=1)

In [ ]:
v=raw_data.columns.values

In [ ]:
raw_data['site_hole']=raw_data.Site+raw_data.Hole
print(raw_data.site_hole.unique())
site_holes=raw_data.site_hole.unique()
first_derivatives=dict(zip(raw_data.site_hole.unique(),np.zeros(raw_data.site_hole.unique().shape)))
km_first_derivatives=dict(zip(raw_data.site_hole.unique(),np.zeros(raw_data.site_hole.unique().shape)))

In [ ]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise (ValueError, "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise (ValueError, "Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise (ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

def first_derivative_smooth(reflectance,wavelength,wl=5,filter='flat'):
    first=int((wl-1)/2)
    smoothed_reflectance = smooth(reflectance,wl,filter)[first:-first]
    return pd.Series(data=np.gradient(smoothed_reflectance,wavelength),index=wavelength)

def first_derivative(reflectance,wavelength):
    return pd.Series(data=np.gradient(reflectance,wavelength),index=wavelength)

In [ ]:
smooth_param=99
def kubelka_munk(x):
    x = x/100
    return ((1-x)**2)/(2*x)

data_columns=['depth-m']+[str(x) for x in np.linspace(400,850,num=226,dtype='int')]

for sh in site_holes:
    print('Working on: ',sh)
    sh_data=raw_data.loc[(raw_data.site_hole==sh),[(x in data_columns) for x in raw_data.columns]]
    sh_kubelka_munk=sh_data
    sh_kubelka_munk.loc[:,'400':'850']=sh_data.loc[:,'400':'850'].applymap(kubelka_munk)
    wavelength=[float(i) for i in sh_data.drop('depth-m',axis=1).columns]
    sh_grad_smooth=sh_data.drop('depth-m',axis=1).apply(first_derivative_smooth,axis=1,raw=True,wavelength=wavelength,wl=smooth_param)
    sh_grad_smooth['depth-m']=sh_data['depth-m']
    first_derivatives[sh]=sh_grad_smooth
    
    sh_grad_km_smooth=sh_kubelka_munk.drop('depth-m',axis=1).apply(first_derivative_smooth,axis=1,raw=True,wavelength=wavelength,wl=smooth_param)
    sh_grad_km_smooth['depth-m']=sh_kubelka_munk['depth-m']
    km_first_derivatives[sh]=sh_grad_km_smooth
   
print('Finished.')

In [ ]:
km_first_derivatives_df=pd.DataFrame.from_dict({(x,y):km_first_derivatives[x][y]
                                                for x in km_first_derivatives.keys()
                                                for y in km_first_derivatives[x].keys()},
                                            orient='columns')
km_first_derivatives_df=km_first_derivatives_df.stack(level=0)
km_first_derivatives_df=km_first_derivatives_df.droplevel(0)
km_first_derivatives_df=km_first_derivatives_df.drop_duplicates()

site_info_DRS=km_first_derivatives_df['depth-m'].reset_index()
#takes index and depth into new data frame and makes them columns
site_info_DRS=site_info_DRS.rename(columns={'index':'site'})
site_info_DRS['label']=site_info_DRS['site']+'-'+site_info_DRS['depth-m'].astype('str')
#adds new column and assigns to data from site_info (astype- convert to string)
site_info_DRS.reset_index(inplace=True)

km_first_derivatives_df=km_first_derivatives_df.drop(('depth-m'),axis=1)
min_value_DRS=km_first_derivatives_df.values.min(axis=None)
km_first_derivatives_shifted=km_first_derivatives_df-min_value_DRS

km_first_derivatives_plus_site_info=km_first_derivatives_df.reset_index()
km_first_derivatives_plus_site_info=km_first_derivatives_plus_site_info.rename(columns={'index':'site'})
km_first_derivatives_plus_site_info=pd.merge(site_info_DRS,km_first_derivatives_plus_site_info,on=['site'])

km_derivatives_coercivity=pd.merge(km_first_derivatives_plus_site_info,coercivity, on=['site','depth-m'])
site_info_DRS_coercivity=km_derivatives_coercivity[['site','depth-m','label']].reset_index()


km_derivatives_coercivity_hysteresis=pd.merge(km_derivatives_coercivity,hysteresis, on=['site','depth-m'])
site_info_DRS_coercivity_hysteresis=km_derivatives_coercivity_hysteresis[['site','depth-m','label']].reset_index()

km_derivatives_coercivity=km_derivatives_coercivity.drop(['index','depth-m','site','label','max_d2','core','section','interval','Unnamed: 0','id'], axis=1)
min_value_DRS_coercivity=km_derivatives_coercivity.values.min(axis=None)
km_derivatives_coercivity_shifted=km_derivatives_coercivity-min_value_DRS_coercivity

km_derivatives_coercivity_hysteresis=km_derivatives_coercivity_hysteresis.drop(['index','depth-m','site','label','max_d2',
    'core_x','section_x','interval_x','Unnamed: 0_x','id_x','Unnamed: 0_y','id_y','core_y','section_y','interval_y','top [cm]','bottom [cm]','lithology principal name','top depth csf-a (m)'], axis=1)
min_value_DRS_coercivity_hysteresis=km_derivatives_coercivity_hysteresis.values.min(axis=None)
km_derivatives_coercivity_hysteresis_shifted=km_derivatives_coercivity_hysteresis-min_value_DRS_coercivity_hysteresis